In [2]:
# 1) IMPORTS & DATA LOADING
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from deepchem.molnet import load_pdbbind
# Qiskit ML pieces
from qiskit.circuit.library import ZFeatureMap
from qiskit_machine_learning.algorithms import VQR
from qiskit.primitives import Estimator
from scipy.optimize import L_BFGS_B

# 2) LOAD DEEPCHEM DATA
# (assumes you've already run your AtomicConvFeaturizer load & MyTransformer steps)
tasks, datasets, transformers = load_pdbbind(
    featurizer=acf,
    save_dir='.',
    data_dir='.',
    pocket=True,
    reload=False,
    set_name='core'
)
train, val, test = datasets
X_train_dc, y_train = train.X, train.y
X_val_dc,   y_val   = val.X,   val.y
X_test_dc,  y_test  = test.X,  test.y

# 3) FLATTEN DeepChem FEATURES → CLASSICAL MATRIX
def flatten_dc_features(X_dc):
    flat = []
    for sample in X_dc:
        # each sample is a tuple/list of 9 arrays (one per feature type)
        arrays = [arr.flatten() for arr in sample]
        flat.append(np.concatenate(arrays))
    return np.vstack(flat)

X_train_flat = flatten_dc_features(X_train_dc)
X_val_flat   = flatten_dc_features(X_val_dc)
X_test_flat  = flatten_dc_features(X_test_dc)

# 4) SCALE + PCA → N‐QUBIT INPUT VECTORS
n_qubits = 6  # pick based on your simulator capacity
# scale into [−π,π] so feature_map rotations stay numeric
fm_scaler = MinMaxScaler(feature_range=(-np.pi, np.pi))
X_all = fm_scaler.fit_transform(
    np.vstack([X_train_flat, X_val_flat, X_test_flat])
)
pca = PCA(n_components=n_qubits)
X_all_pca = pca.fit_transform(X_all)

# split back
N_train = X_train_flat.shape[0]
N_val   = X_val_flat.shape[0]
X_train_pca = X_all_pca[:N_train]
X_val_pca   = X_all_pca[N_train:N_train+N_val]
X_test_pca  = X_all_pca[N_train+N_val:]

# scale targets y into [−1,+1] for single‐qubit range
y_scaler = MinMaxScaler(feature_range=(-1, 1))
y_all = y_scaler.fit_transform(
    np.vstack([y_train.reshape(-1,1), y_val.reshape(-1,1), y_test.reshape(-1,1)])
)
y_train_scaled = y_all[:N_train].ravel()
y_val_scaled   = y_all[N_train:N_train+N_val].ravel()
y_test_scaled  = y_all[N_train+N_val:].ravel()

# 5) BUILD THE QCNN CIRCUIT
# 5.1 Feature map
feature_map = ZFeatureMap(num_qubits=n_qubits, reps=1)

# 5.2 A simple “1‐conv + 1‐pool” QCNN block for illustration
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def conv_instruction(n, prefix):
    params = ParameterVector(prefix, length=3*n)
    qc = QuantumCircuit(n, name="Conv")
    idx = 0
    for q1,q2 in zip(range(0,n,2), range(1,n,2)):
        sub = QuantumCircuit(2)
        sub.rz(-np.pi/2, 1)
        sub.cx(1,0)
        sub.rz(params[idx],   0)
        sub.ry(params[idx+1], 1)
        sub.cx(0,1)
        sub.ry(params[idx+2], 1)
        qc.compose(sub, [q1,q2], inplace=True)
        qc.barrier()
        idx += 3
    return qc.to_instruction()

def pool_instruction(srcs, sinks, prefix):
    params = ParameterVector(prefix, length=3*len(srcs))
    n = len(srcs)+len(sinks)
    qc = QuantumCircuit(n, name="Pool")
    idx = 0
    for s,t in zip(srcs, sinks):
        sub = QuantumCircuit(2)
        sub.rz(-np.pi/2, 1)
        sub.cx(1,0)
        sub.rz(params[idx],   0)
        sub.ry(params[idx+1], 1)
        sub.cx(0,1)
        sub.ry(params[idx+2], 1)
        qc.compose(sub, [s,t], inplace=True)
        qc.barrier()
        idx += 3
    return qc.to_instruction()

# Assemble: fmap → conv → pool → measure Z⊗… at end
qc_cnn = QuantumCircuit(n_qubits)
qc_cnn.compose(feature_map, range(n_qubits), inplace=True)
qc_cnn.append(conv_instruction(n_qubits, "c1"), range(n_qubits))
# example pooling halving qubits [0→n/2],[1→n/2+1],…
src = list(range(0,n_qubits,2))
snk = list(range(1,n_qubits,2))
qc_cnn.append(pool_instruction(src, snk, "p1"), range(n_qubits))
# now only n_qubits/2 logical remain; you’d repeat conv+pool until 1 remains

# 6) WRAP IN A VQR & TRAIN
vqr = VQR(
    feature_map=feature_map,
    ansatz=qc_cnn,
    optimizer=L_BFGS_B(maxiter=150),
    estimator=Estimator(),
)

# fit on train, validate on val
vqr.fit(X_train_pca, y_train_scaled)
y_val_pred_scaled = vqr.predict(X_val_pca)
y_val_pred = y_scaler.inverse_transform(y_val_pred_scaled.reshape(-1,1)).ravel()

# 7) EVALUATE ON TEST
y_test_pred_scaled = vqr.predict(X_test_pca)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled.reshape(-1,1)).ravel()

# Now you have y_test_pred in the original binding‐energy units!


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/site-packages/traitlets/config/appl

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sanskriti/miniconda3/envs/atomic/lib/python3.10/site-packages/traitlets/config/appl

AttributeError: _UFUNC_API not found

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle